<a href="https://colab.research.google.com/github/JuanLara18/Actuarial-DataMined-Provisiones/blob/main/notebooks/Project_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The data can be download [here](https://www.casact.org/publications-research/research/research-resources/loss-reserving-data-pulled-naic-schedule-p).

# Data Understanding

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

data = pd.read_csv("wkcomp_pos.csv")

In [22]:
# Display basic information about the dataset
print("Dataset Information:")
print("Number of Records:", data.shape[0])
print("Number of Attributes:", data.shape[1])

Dataset Information:
Number of Records: 13200
Number of Attributes: 13


In [23]:
# Display data types of attributes
print("\nData Types of Attributes:")
print(data.dtypes)


Data Types of Attributes:
GRCODE                int64
GRNAME               object
AccidentYear          int64
DevelopmentYear       int64
DevelopmentLag        int64
IncurLoss_D           int64
CumPaidLoss_D         int64
BulkLoss_D            int64
EarnedPremDIR_D       int64
EarnedPremCeded_D     int64
EarnedPremNet_D       int64
Single                int64
PostedReserve97_D     int64
dtype: object


In [24]:
# Check unique values and coding schemes for specific attributes
print("\nCoding Schemes:")
print("Number of unique values in 'GRCODE':", len(data['GRCODE'].unique()))
print("Number of unique values in 'GRNAME':", len(data['GRNAME'].unique()))
print("Number of unique values in 'AccidentYear':", len(data['AccidentYear'].unique()))
print("Number of unique values in 'DevelopmentYear':", len(data['DevelopmentYear'].unique()))


Coding Schemes:
Number of unique values in 'GRCODE': 132
Number of unique values in 'GRNAME': 132
Number of unique values in 'AccidentYear': 10
Number of unique values in 'DevelopmentYear': 19


# Data Preparation and Statistics Related

In [25]:
# Extract de information in DataFrames
Important_Variables = data.columns.delete([0,1,2,3,4,-1,-2,-3,-4,-5])
DataComplete = {column : [] for column in Important_Variables}
size = data.shape[0]

for Variable in Important_Variables:
  count = 0
  while(size - 100*count > 1):
    base_Variable = np.zeros((10,10))
    for i in range(10):
      for j in range(10):
        base_Variable[i][j] = data[Variable][i*10+j + 10*count]
    count = count + 1
    DataComplete[Variable].append(pd.DataFrame(base_Variable))

In [4]:
# Construct the Triangles
def IncompleteDataFrame(x):
  matrix = x.values
  for i in range(9,0,-1):
    for j in range(10-i,10,1):
        matrix[i][j] = 0
  return pd.DataFrame(matrix)

Triangles = {}
for variable, array in DataComplete.items():
  Triangles[variable] = [IncompleteDataFrame(x) for x in array]

In [5]:
# Find the basics statistics component by component for each variable
Means = {}
for variable, array in Triangles.items():
  Means[variable] = pd.concat(array).groupby(level=0).mean()

Var = {}
for variable, array in Triangles.items():
  Var[variable] = pd.concat(array).groupby(level=0).var()

Min = {}
for variable, array in Triangles.items():
  Min[variable] = pd.concat(array).groupby(level=0).min()

Max = {}
for variable, array in Triangles.items():
  Max[variable] = pd.concat(array).groupby(level=0).max()

Median = {}
for variable, array in Triangles.items():
  Median[variable] = pd.concat(array).groupby(level=0).median()

In [6]:
# Define the function to plot the series
def DrawGraph(x, name):
  matrix = x.values
  plt.figure(figsize=(9, 7))
  plt.plot()

  for i in range(10):
    val = matrix[i]
    for j in range(i):
      val = val[:-1]
    rang = range(1988,1988+len(val))
    plt.plot(rang, val, marker='o', linestyle='-', label=1988+i)

  plt.xlabel('Years')
  plt.ylabel('Value')
  plt.title(name)

  plt.savefig(name, dpi=300, bbox_inches='tight')

  plt.legend()

In [ ]:
# Export the graphs of the basis statistics
for var in Important_Variables:
  DrawGraph(Means[var], 'Mean: ' + var)

for var in Important_Variables:
  DrawGraph(Var[var], 'Variance: ' + var)

for var in Important_Variables:
  DrawGraph(Median[var], 'Median: ' + var)

In [ ]:
# PCA
from sklearn.decomposition import PCA

def dataframe_to_pca(dataframes, n_components):

  X = np.array([])
  for i in range(len(dataframes)):
    flattened_vector = dataframes[i].values.flatten()
    X = np.append(X, flattened_vector)

  X = X.reshape(len(dataframes), -1)

  pca = PCA(n_components=n_components)
  pca_result = pca.fit_transform(X)

  return pca_result

In [ ]:
# Outliers



# Models

## Chain Ladder technique

In [68]:
def chain_ladder_deterministic(data):

    data = data.values.tolist()
    for i in range(1,10):
      for j in range(i):
        data[i].pop()

    development_factors = []
    cumulative_factors = []

    # Calcular los factores de desarrollo
    for i in range(1, len(data)):
        development_factor = sum(data[i])/sum(data[i - 1])
        development_factors.append(development_factor)

    # Calcular los factores acumulados
    cumulative_factor = 1
    for factor in development_factors:
        cumulative_factor *= factor
        cumulative_factors.append(cumulative_factor)

    # Proyectar reclamaciones futuras
    ultimate_claim = sum(data[0]) * cumulative_factors[-1]
    projected_claims = [ultimate_claim * factor for factor in cumulative_factors]

    print(ultimate_claim)
    print(projected_claims)

    # Completar el triángulo de datos con las proyecciones
    full_data = data.copy()
    for i in range(len(data), len(data) + len(projected_claims)):
        print("i:",i)
        print(projected_claims[i - len(data)])
        full_data.append([projected_claims[i - len(data)]])

    return full_data

# Ejemplo de uso
data = Triangles['IncurLoss_D'][0]

full_data = chain_ladder_deterministic(data)

pd.DataFrame(full_data)


6725.0
[5232.050656198458, 4353.024855340758, 3694.044270392738, 2195.8031273386496, 1010.0103039662924, 785.5539762068679, 551.5443966423593, 200.67009304811094, 12.90331761180795]
i: 10
5232.050656198458
i: 11
4353.024855340758
i: 12
3694.044270392738
i: 13
2195.8031273386496
i: 14
1010.0103039662924
i: 15
785.5539762068679
i: 16
551.5443966423593
i: 17
200.67009304811094
i: 18
12.90331761180795


,0,1,2,3,4,5,6,7,8,9
0,367404.000000,362988.0,347288.0,330648.0,354690.0,350092.0,346808.0,349124.0,348157.0,347762.0
1,336928.000000,316483.0,278496.0,303033.0,299496.0,295061.0,299251.0,297492.0,300620.0,NaN
2,289198.000000,311381.0,277980.0,277732.0,276563.0,278067.0,276704.0,281101.0,NaN,NaN
3,297174.000000,277209.0,269739.0,272666.0,271318.0,267578.0,269592.0,NaN,NaN,NaN
4,181796.000000,205079.0,199106.0,187833.0,185663.0,184940.0,NaN,NaN,NaN,NaN
5,114807.000000,114774.0,101460.0,98430.0,96930.0,NaN,NaN,NaN,NaN,NaN
6,107934.000000,107569.0,97730.0,96185.0,NaN,NaN,NaN,NaN,NaN,NaN
7,100686.000000,94456.0,92314.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,53381.000000,51205.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,6725.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
DataComplete["IncurLoss_D"][0]

,0,1,2,3,4,5,6,7,8,9
0,367404.0,362988.0,347288.0,330648.0,354690.0,350092.0,346808.0,349124.0,348157.0,347762.0
1,336928.0,316483.0,278496.0,303033.0,299496.0,295061.0,299251.0,297492.0,300620.0,296626.0
2,289198.0,311381.0,277980.0,277732.0,276563.0,278067.0,276704.0,281101.0,278023.0,279032.0
3,297174.0,277209.0,269739.0,272666.0,271318.0,267578.0,269592.0,267745.0,266677.0,264467.0
4,181796.0,205079.0,199106.0,187833.0,185663.0,184940.0,184507.0,184705.0,183888.0,181261.0
5,114807.0,114774.0,101460.0,98430.0,96930.0,96121.0,98085.0,93795.0,93753.0,95363.0
6,107934.0,107569.0,97730.0,96185.0,96124.0,95821.0,95722.0,95496.0,96374.0,95766.0
7,100686.0,94456.0,92314.0,92241.0,92929.0,92214.0,92107.0,92362.0,91470.0,93060.0
8,53381.0,51205.0,51087.0,51015.0,50226.0,50343.0,50060.0,49564.0,49739.0,49459.0
9,6725.0,6663.0,6473.0,2917.0,2933.0,3103.0,3002.0,2909.0,2907.0,2909.0


In [60]:
for i in range(1,10):
  print(i)

1
2
3
4
5
6
7
8
9


In [48]:
pd.DataFrame(full_data)

,0,1,2,3,4,5,6,7,8,9
0,367404.000000,362988.0,347288.0,330648.0,354690.0,350092.0,346808.0,349124.0,348157.0,347762.0
1,336928.000000,316483.0,278496.0,303033.0,299496.0,295061.0,299251.0,297492.0,300620.0,0.0
2,289198.000000,311381.0,277980.0,277732.0,276563.0,278067.0,276704.0,281101.0,0.0,0.0
3,297174.000000,277209.0,269739.0,272666.0,271318.0,267578.0,269592.0,0.0,0.0,0.0
4,181796.000000,205079.0,199106.0,187833.0,185663.0,184940.0,0.0,0.0,0.0,0.0
5,114807.000000,114774.0,101460.0,98430.0,96930.0,0.0,0.0,0.0,0.0,0.0
6,107934.000000,107569.0,97730.0,96185.0,0.0,0.0,0.0,0.0,0.0,0.0
7,100686.000000,94456.0,92314.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,53381.000000,51205.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,6725.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
Triangles.keys()

dict_keys(['IncurLoss_D', 'CumPaidLoss_D', 'BulkLoss_D'])

In [42]:
Triangles['IncurLoss_D'][0].values.tolist()

[[367404.0,
  362988.0,
  347288.0,
  330648.0,
  354690.0,
  350092.0,
  346808.0,
  349124.0,
  348157.0,
  347762.0],
 [336928.0,
  316483.0,
  278496.0,
  303033.0,
  299496.0,
  295061.0,
  299251.0,
  297492.0,
  300620.0,
  0.0],
 [289198.0,
  311381.0,
  277980.0,
  277732.0,
  276563.0,
  278067.0,
  276704.0,
  281101.0,
  0.0,
  0.0],
 [297174.0,
  277209.0,
  269739.0,
  272666.0,
  271318.0,
  267578.0,
  269592.0,
  0.0,
  0.0,
  0.0],
 [181796.0,
  205079.0,
  199106.0,
  187833.0,
  185663.0,
  184940.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [114807.0, 114774.0, 101460.0, 98430.0, 96930.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [107934.0, 107569.0, 97730.0, 96185.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [100686.0, 94456.0, 92314.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [53381.0, 51205.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [6725.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [27]:
df = DataComplete["IncurLoss_D"][0]

df.to_excel("CompleteBox.xlsx", index="False")

In [26]:
DataComplete["IncurLoss_D"][0]

,0,1,2,3,4,5,6,7,8,9
0,367404.0,362988.0,347288.0,330648.0,354690.0,350092.0,346808.0,349124.0,348157.0,347762.0
1,336928.0,316483.0,278496.0,303033.0,299496.0,295061.0,299251.0,297492.0,300620.0,296626.0
2,289198.0,311381.0,277980.0,277732.0,276563.0,278067.0,276704.0,281101.0,278023.0,279032.0
3,297174.0,277209.0,269739.0,272666.0,271318.0,267578.0,269592.0,267745.0,266677.0,264467.0
4,181796.0,205079.0,199106.0,187833.0,185663.0,184940.0,184507.0,184705.0,183888.0,181261.0
5,114807.0,114774.0,101460.0,98430.0,96930.0,96121.0,98085.0,93795.0,93753.0,95363.0
6,107934.0,107569.0,97730.0,96185.0,96124.0,95821.0,95722.0,95496.0,96374.0,95766.0
7,100686.0,94456.0,92314.0,92241.0,92929.0,92214.0,92107.0,92362.0,91470.0,93060.0
8,53381.0,51205.0,51087.0,51015.0,50226.0,50343.0,50060.0,49564.0,49739.0,49459.0
9,6725.0,6663.0,6473.0,2917.0,2933.0,3103.0,3002.0,2909.0,2907.0,2909.0


## Lineal

In [28]:
ejemplo = Triangles['IncurLoss_D'][0]

In [29]:
ejemplo

,0,1,2,3,4,5,6,7,8,9
0,367404.0,362988.0,347288.0,330648.0,354690.0,350092.0,346808.0,349124.0,348157.0,347762.0
1,336928.0,316483.0,278496.0,303033.0,299496.0,295061.0,299251.0,297492.0,300620.0,0.0
2,289198.0,311381.0,277980.0,277732.0,276563.0,278067.0,276704.0,281101.0,0.0,0.0
3,297174.0,277209.0,269739.0,272666.0,271318.0,267578.0,269592.0,0.0,0.0,0.0
4,181796.0,205079.0,199106.0,187833.0,185663.0,184940.0,0.0,0.0,0.0,0.0
5,114807.0,114774.0,101460.0,98430.0,96930.0,0.0,0.0,0.0,0.0,0.0
6,107934.0,107569.0,97730.0,96185.0,0.0,0.0,0.0,0.0,0.0,0.0
7,100686.0,94456.0,92314.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,53381.0,51205.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,6725.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
zetas = np.zeros(10)

TypeError: ignored

In [32]:
zetas

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])